# Data Cleaning Results Analysis

This notebook analyzes the results of our ML-based data cleaning pipeline, comparing the original and cleaned datasets to evaluate the effectiveness of our cleaning process.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
from scipy import stats
import yaml

# Set style for visualizations
plt.style.use('seaborn')
sns.set_palette('husl')

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)  # Format floats to 3 decimal places

In [ ]:
# Load the original and cleaned datasets
try:
    original_df = pd.read_csv('../data/raw/input_dataset.csv')
    cleaned_df = pd.read_csv('../data/processed/cleaned_dataset.csv')
except FileNotFoundError as e:
    raise FileNotFoundError("Could not find one of the required CSV files. Please ensure both input_dataset.csv and cleaned_dataset.csv exist in their respective directories.") from e

# Load configuration
try:
    with open('../configs/cleaning_config.yaml', 'r') as f:
        config = yaml.safe_load(f)
except FileNotFoundError:
    raise FileNotFoundError("Could not find cleaning_config.yaml. Please ensure it exists in the configs directory.")
except yaml.YAMLError as e:
    raise yaml.YAMLError("Error parsing cleaning_config.yaml. Please ensure it is valid YAML format.") from e

## 1. Basic Statistics Comparison

In [ ]:
def calculate_basic_stats(original_df, cleaned_df):
    # Handle potential null values in text and hashtags columns
    orig_text_len = original_df['text'].fillna('').str.len()
    clean_text_len = cleaned_df['text'].fillna('').str.len()
    orig_hashtags = original_df['hashtags'].fillna('').str.count('#')
    clean_hashtags = cleaned_df['hashtags'].fillna('').str.count('#')
    
    stats_dict = {
        'Metric': [
            'Total Records',
            'Records Removed',
            'Removal Rate (%)',
            'Average Text Length',
            'Median Text Length', 
            'Average Hashtags per Record',
            'Unique Country Codes',
            'Missing Values (Total)'
        ],
        'Original': [
            len(original_df),
            0,
            0,
            orig_text_len.mean(),
            orig_text_len.median(),
            orig_hashtags.mean(),
            original_df['country_code'].nunique(),
            original_df.isnull().sum().sum()
        ],
        'Cleaned': [
            len(cleaned_df),
            len(original_df) - len(cleaned_df),
            ((len(original_df) - len(cleaned_df)) / len(original_df) * 100),
            clean_text_len.mean(),
            clean_text_len.median(), 
            clean_hashtags.mean(),
            cleaned_df['country_code'].nunique(),
            cleaned_df.isnull().sum().sum()
        ]
    }
    
    # Create DataFrame and ensure numeric columns are rounded
    stats_df = pd.DataFrame(stats_dict)
    numeric_columns = ['Original', 'Cleaned']
    stats_df[numeric_columns] = stats_df[numeric_columns].round(2)
    
    return stats_df

basic_stats = calculate_basic_stats(original_df, cleaned_df)
basic_stats

## 2. Text Quality Analysis

In [ ]:
def analyze_text_quality(df, column='text'):
    """Analyze text quality metrics
    
    Args:
        df (pd.DataFrame): DataFrame containing text data
        column (str): Name of the text column to analyze
        
    Returns:
        pd.Series: Series containing text quality metrics
    """
    # Handle null values safely
    text_series = df[column].fillna('')
    
    # Calculate metrics with error handling
    try:
        total_words = text_series.str.split().str.len().sum()
        unique_words = len(set(' '.join(text_series[text_series != '']).split()))
        avg_words = text_series.str.split().str.len().mean()
        length_std = text_series.str.len().std()
        empty_texts = df[column].isna().sum()
        short_texts = (text_series.str.len() < 10).sum()
        
        metrics = {
            'Total Words': total_words,
            'Unique Words': unique_words,
            'Average Words per Text': round(avg_words, 2),
            'Text Length Std Dev': round(length_std, 2),
            'Empty Texts': empty_texts,
            'Short Texts (<10 chars)': short_texts
        }
        
        return pd.Series(metrics)
    
    except Exception as e:
        print(f"Error calculating metrics: {str(e)}")
        return pd.Series()

# Compare text quality
original_quality = analyze_text_quality(original_df)
cleaned_quality = analyze_text_quality(cleaned_df)

# Calculate percentage changes, handling division by zero
quality_comparison = pd.DataFrame({
    'Original': original_quality,
    'Cleaned': cleaned_quality
})

# Calculate percentage change safely
quality_comparison['Change (%)'] = quality_comparison.apply(
    lambda row: round(((row['Cleaned'] - row['Original']) / row['Original'] * 100), 2) 
    if row['Original'] != 0 else float('inf'),
    axis=1
)

quality_comparison

## 3. Visualization of Changes

In [ ]:
def plot_text_length_distribution(original_df, cleaned_df):
    # Handle potential missing or invalid data
    if 'text' not in original_df.columns or 'text' not in cleaned_df.columns:
        print("Error: 'text' column not found in dataframes")
        return
        
    # Create figure with larger size for better readability
    plt.figure(figsize=(15, 8))
    
    # Calculate text lengths, handling NaN values
    orig_lengths = original_df['text'].fillna('').str.len()
    clean_lengths = cleaned_df['text'].fillna('').str.len()
    
    # Plot distributions with reasonable bin size
    bins = min(50, int(max(orig_lengths.max(), clean_lengths.max()) / 20))
    
    # Plot original distribution
    sns.histplot(data=orig_lengths, 
                label='Original', 
                alpha=0.5,
                bins=bins,
                color='blue')
    
    # Plot cleaned distribution
    sns.histplot(data=clean_lengths,
                label='Cleaned',
                alpha=0.5, 
                bins=bins,
                color='green')
    
    plt.title('Text Length Distribution Before and After Cleaning', pad=20)
    plt.xlabel('Text Length (characters)', labelpad=10)
    plt.ylabel('Count', labelpad=10)
    plt.legend(title='Dataset')
    
    # Add grid for better readability
    plt.grid(True, alpha=0.3)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    plt.show()

# Generate the plot
plot_text_length_distribution(original_df, cleaned_df)

In [ ]:
def plot_country_distribution(original_df, cleaned_df):
    # Handle potential missing data
    if 'country_code' not in original_df.columns or 'country_code' not in cleaned_df.columns:
        print("Error: 'country_code' column not found in dataframes")
        return
        
    # Create figure
    fig = go.Figure()
    
    # Calculate value counts, handling NaN values
    orig_counts = original_df['country_code'].fillna('Unknown').value_counts()
    clean_counts = cleaned_df['country_code'].fillna('Unknown').value_counts()
    
    # Add original distribution
    fig.add_trace(go.Bar(
        x=orig_counts.index,
        y=orig_counts.values,
        name='Original',
        marker_color='lightblue'
    ))
    
    # Add cleaned distribution
    fig.add_trace(go.Bar(
        x=clean_counts.index,
        y=clean_counts.values,
        name='Cleaned',
        marker_color='lightgreen'
    ))
    
    # Update layout with more details
    fig.update_layout(
        title={
            'text': 'Country Code Distribution Before and After Cleaning',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title='Country Code',
        yaxis_title='Count',
        barmode='group',
        bargap=0.2,
        bargroupgap=0.1,
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99
        ),
        plot_bgcolor='white'
    )
    
    # Add gridlines for better readability
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
    
    fig.show()

# Generate the plot
plot_country_distribution(original_df, cleaned_df)

## 4. Hashtag Analysis

In [ ]:
def analyze_hashtags(df, column='hashtags'):
    # Handle empty dataframe case
    if df.empty:
        return pd.Series({
            'Total Hashtags': 0,
            'Unique Hashtags': 0, 
            'Average Hashtags per Record': 0,
            'Records with Hashtags (%)': 0
        })
    
    # Split hashtags and create a list of all hashtags
    # Handle potential NaN values and ensure proper string splitting
    all_hashtags = []
    for tags in df[column].dropna():
        if isinstance(tags, str):
            # Split on whitespace and filter out empty strings
            hashtags = [tag.strip() for tag in tags.split() if tag.strip()]
            all_hashtags.extend(hashtags)
    
    # Count unique hashtags
    unique_hashtags = len(set(all_hashtags))
    
    # Get top hashtags
    top_hashtags = pd.Series(all_hashtags).value_counts().head(10)
    
    # Create word cloud only if there are hashtags
    if all_hashtags:
        wordcloud = WordCloud(width=800, height=400, 
                            background_color='white',
                            min_font_size=10,
                            max_font_size=50)
        wordcloud.generate(' '.join(all_hashtags))
        
        # Plotting
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
        
        # Plot top hashtags
        top_hashtags.plot(kind='barh', ax=ax1)
        ax1.set_title('Top 10 Hashtags')
        ax1.set_xlabel('Frequency')
        ax1.set_ylabel('Hashtags')
        
        # Plot word cloud
        ax2.imshow(wordcloud, interpolation='bilinear')
        ax2.axis('off')
        ax2.set_title('Hashtag Word Cloud')
        
        plt.tight_layout()
        plt.show()
    else:
        print("No hashtags found in the dataset")
    
    # Calculate statistics
    total_records = len(df)
    records_with_hashtags = df[column].notna().sum()
    
    return pd.Series({
        'Total Hashtags': len(all_hashtags),
        'Unique Hashtags': unique_hashtags,
        'Average Hashtags per Record': len(all_hashtags) / total_records if total_records > 0 else 0,
        'Records with Hashtags (%)': (records_with_hashtags / total_records * 100).round(2) if total_records > 0 else 0
    })

print("Original Dataset Hashtag Analysis:")
original_hashtag_stats = analyze_hashtags(original_df)
print(original_hashtag_stats)

print("\nCleaned Dataset Hashtag Analysis:")
cleaned_hashtag_stats = analyze_hashtags(cleaned_df)
print(cleaned_hashtag_stats)

## 5. Development Status Changes

In [ ]:
def analyze_development_status(original_df, cleaned_df):
    # Check if development_status column exists in both dataframes
    if 'development_status' not in original_df.columns or 'development_status' not in cleaned_df.columns:
        raise ValueError("development_status column not found in one or both dataframes")
        
    # Create comparison DataFrame
    comparison = pd.DataFrame({
        'Original': original_df['development_status'].value_counts(normalize=True) * 100,
        'Cleaned': cleaned_df['development_status'].value_counts(normalize=True) * 100
    }).round(2)
    
    # Calculate changes
    comparison['Change (%)'] = (comparison['Cleaned'] - comparison['Original']).round(2)
    
    # Plotting
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Pie charts with better formatting
    original_df['development_status'].value_counts().plot(
        kind='pie', 
        autopct='%1.1f%%', 
        ax=ax1, 
        title='Original Distribution',
        labeldistance=1.1)
    ax1.set_ylabel('')  # Remove unnecessary y-label
    
    cleaned_df['development_status'].value_counts().plot(
        kind='pie', 
        autopct='%1.1f%%', 
        ax=ax2, 
        title='Cleaned Distribution',
        labeldistance=1.1)
    ax2.set_ylabel('')  # Remove unnecessary y-label
    
    plt.tight_layout()
    plt.show()
    
    # Add total counts
    comparison['Original Count'] = original_df['development_status'].value_counts()
    comparison['Cleaned Count'] = cleaned_df['development_status'].value_counts()
    
    return comparison

# Analyze development status changes
development_status_comparison = analyze_development_status(original_df, cleaned_df)
display(development_status_comparison)